In [0]:
%sql
-- SILVER LAYER TRANSFORMATION LOGIC

-- Step 1: Select Catalog and Schema 
-- Set the working environment using SQL, pointing to the relevant catalog and schema 
-- where the bronze table will be saved.

USE CATALOG vsqproject;
USE SCHEMA bronze;

In [0]:
# source_table = dbutils.widgets.get("source_table")
# sink_table = dbutils.widgets.get("sink_table")
source_table = "raw_products"
sink_table = "products_silver"

In [0]:
# Step 2: Read Raw Bronze Layer Data
# Load the raw 'products' Delta table from the Bronze layer into a DataFrame.
# This acts as our source data for the Silver layer transformation process.

df= spark.table(source_table)
# display(df)

In [0]:
from pyspark.sql.functions import col, current_timestamp,trim,mean

# Step 3: Remove rows with nulls in essential columns
# Filter: not null AND not empty (after trimming spaces)
filteredDf = df.filter(col("product_id").isNotNull() & (trim(col("product_id")) != ""))
df_silver = filteredDf.withColumn("price", filteredDf["price"].cast("double"))

# Step 4: Replace remaining nulls with default values
# If any nulls are still present:
# - Replace missing price with the average price from the dataset
# - Replace missing product_name with 'Unknown'
# - Replace missing category with 'Miscellaneous'
mean_price_row = df_silver.select(mean("price")).alias("avg_price").first()[0] or 0.0

# Replace nulls with defaults
df_silver = df_silver.fillna({
    "price": mean_price_row,
    "productname": "Unknown",
    "brand": "UnknownBrand",
    "category": "Miscellaneous"
})

# Step 5: Drop duplicate rows based on product_id
# This ensures each product appears only once in the dataset.
df_silver = df_silver.dropDuplicates(['product_id'])


# Step 6: Filter out invalid prices
# Remove any products where price is zero or negative, since they are considered invalid for business reporting.
df_silver = df_silver.filter(col('price') > 0)

# Step 7: Add last updated timestamp
# Add a new column 'last_updated' to capture the date and time when the data was processed.
df_silver = df_silver.withColumn("last_updated", current_timestamp())


# Step 8 : Save transformed data to the Silver layer
# Write the data as a Delta table in the 'silver' schema, overwriting any previous version.
df_silver.write.format("delta").option("mergeSchema", "true").partitionBy('category').mode("overwrite").saveAsTable(f"vsqproject.silver.{sink_table}")

In [0]:
# Step 11: Verify the Silver Table Load
# Run a query to ensure that the Silver layer data was written successfully.

# df = spark.sql(f"select * from vsqproject.silver.{sink_table}")
# df.printSchema()
# display(df)

Summary
=============

In this Silver layer process:

Source: Data is read from the Bronze layer Delta table.

**Cleaning:**
Removed records where product_id was null or empty (after trimming spaces).

Filled missing values:

price → replaced with average price from dataset.

product_name → replaced with "Unknown".

category → replaced with "Miscellaneous".

Dropped duplicate products based on product_id.

Removed records with non-positive prices (≤ 0).

**Enrichment**:

Added last_updated timestamp to capture processing time.

**Output:**
Saved the cleaned and enriched dataset as vsqproject.silver.Products_silver in Delta format.

This ensures the Silver table contains accurate, validated, and timestamped data, ready for analytics, reporting, or further transformation in the Gold layer.

